In [1]:
# this notebook will give you the appotunity to work with dynamic data

In [2]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import trimboth
import ast
import time
import datetime
import geopy
from tqdm import tqdm
import geopy.distance
from tqdm._tqdm_notebook import tqdm_notebook
tqdm.pandas(desc="Bar")

In [3]:
import plotly.offline as pyo
import chart_studio.plotly as chspy
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.express as px

init_notebook_mode(connected=True)

In [4]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import seaborn as sns; sns.set()
import csv

In [5]:
flats = pd.read_csv('final.csv', index_col=0)

In [25]:
spb = flats[(flats.trade_type == 'sale')&\
            ((flats.city == 'Санкт-Петербург')| (flats.city == 'Ленинградская область'))]

In [26]:
spb['date'] = spb.date_of_adding_to_db

In [20]:
spb.head()

,Building_year,Number_of_rooms,_id,active,address,bathroom_number,bathroom_separated,ceiling_type,central_heating,cian_id,...,visitors,whole_storey_number,windows_to_street,сeiling_height,trade_type,New_building,quarter,city,date,min_passive_date
0,2014.0,1.0,ObjectId(5e579c1b5aaf06a3bbfabfa0),True,Ленинградская область Всеволожский район Бугры...,1.0,No,Железобетонные,Yes,225953134.0,...,NaN,17.0,No,2.0,sale,0.0,NaN,Ленинградская область,2020-02-27,NaN
1,NaN,NaN,ObjectId(5e579e9a256731823afabfd8),True,Ленинградская область Кировский район Мгинское...,NaN,NaN,NaN,NaN,225953318.0,...,NaN,"[null,null]",NaN,NaN,sale,0.0,NaN,Ленинградская область,2020-02-27,NaN
2,1986.0,NaN,ObjectId(5e579eb0089b1f8c93fabfea),True,Ленинградская область Выборгский район Рощино ...,NaN,NaN,NaN,NaN,225953324.0,...,NaN,2.0,NaN,NaN,sale,0.0,NaN,Ленинградская область,2020-02-27,NaN
3,2009.0,3.0,ObjectId(5e57a1a4089b1f8c93fac038),True,Санкт-Петербург Приморский Юнтолово Шаврова 13к1,1.0,Yes,Железобетонные,Yes,225953549.0,...,NaN,16.0,No,2.0,sale,0.0,NaN,Санкт-Петербург,2020-02-27,NaN
4,1976.0,NaN,ObjectId(5e57a1a9550a1f3209fac030),True,Санкт-Петербург Красногвардейский Ржевка Рево...,1.0,Yes,Железобетонные,Yes,225953547.0,...,NaN,9.0,NaN,2.0,sale,0.0,NaN,Санкт-Петербург,2020-02-27,NaN


In [21]:
#made list of unique dates
date_list = spb.date.unique()

In [22]:
date_list

array(['2020-02-27', '2020-02-28', '2020-02-29', '2020-03-01',
       '2020-03-02', '2020-03-03', '2020-03-04', '2020-03-05',
       '2020-03-06', '2020-03-07', '2020-03-08', '2020-03-09',
       '2020-03-10', '2020-03-11', '2020-03-12', '2020-03-13',
       '2020-03-14', '2020-03-15', '2020-03-18', '2020-03-19',
       '2020-03-20', '2020-03-21', '2020-03-22', '2020-03-24',
       '2020-03-26', '2020-03-27', '2020-03-28', '2020-03-29',
       '2020-03-30', '2020-03-31', '2020-04-01', '2020-04-02',
       '2020-04-03', '2020-04-04', '2020-04-05', '2020-04-07',
       '2020-04-08', '2020-04-09', '2020-04-10', '2020-04-12',
       '2020-04-13', '2020-04-14', '2020-04-15', '2020-04-16',
       '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-20',
       '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24',
       '2020-04-25', '2020-04-26'], dtype=object)

In [27]:
# for every cian_id made column which tells when ads became passive
spb = pd.merge(spb, spb[spb.active==False].groupby('cian_id')['date'].min().reset_index().rename(columns={'date':'min_passive_date'}),
               on='cian_id', how='left')

In [31]:
#for a certain day found max active date
max_active_ads = spb[((date_list[53]<spb.min_passive_date)| pd.isnull(spb.min_passive_date)) & \
   (spb.active==True)].groupby('cian_id')['date'].max().reset_index()

In [32]:
max_active_ads

,cian_id,date
0,225953134.0,2020-04-15
1,225953318.0,2020-04-15
2,225953324.0,2020-04-15
3,225953452.0,2020-04-15
4,225953486.0,2020-04-15
...,...,...
2934,228251472.0,2020-04-26
2935,228251473.0,2020-04-26
2936,228251475.0,2020-04-26
2937,228251552.0,2020-04-26


In [30]:
#made merge 
# pd.merge(spb, max_active_ads, on=['cian_id', 'date']).head()

In [12]:
date_dict = {}

In [33]:
for date in tqdm(date_list):
    max_active_ads = spb[((date<spb.min_passive_date)| pd.isnull(spb.min_passive_date)) & \
   (spb.active==True)].groupby('cian_id')['date'].max().reset_index()
    date_dict[pd.to_datetime(date)] = pd.merge(spb, max_active_ads, on=['cian_id', 'date'])

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 54/54 [00:30<00:00,  1.80it/s]


In [34]:
for i in date_dict:
    print(i, date_dict[i].cian_id.count())

2020-02-27 00:00:00 3563
2020-02-28 00:00:00 3563
2020-02-29 00:00:00 3563
2020-03-01 00:00:00 3563
2020-03-02 00:00:00 3563
2020-03-03 00:00:00 3563
2020-03-04 00:00:00 3563
2020-03-05 00:00:00 3563
2020-03-06 00:00:00 3563
2020-03-07 00:00:00 3563
2020-03-08 00:00:00 3563
2020-03-09 00:00:00 3563
2020-03-10 00:00:00 3563
2020-03-11 00:00:00 3563
2020-03-12 00:00:00 3563
2020-03-13 00:00:00 3560
2020-03-14 00:00:00 3560
2020-03-15 00:00:00 3560
2020-03-18 00:00:00 3560
2020-03-19 00:00:00 3560
2020-03-20 00:00:00 3560
2020-03-21 00:00:00 3560
2020-03-22 00:00:00 3560
2020-03-24 00:00:00 3560
2020-03-26 00:00:00 3560
2020-03-27 00:00:00 3560
2020-03-28 00:00:00 3560
2020-03-29 00:00:00 3560
2020-03-30 00:00:00 3560
2020-03-31 00:00:00 3560
2020-04-01 00:00:00 3560
2020-04-02 00:00:00 3560
2020-04-03 00:00:00 3560
2020-04-04 00:00:00 3560
2020-04-05 00:00:00 3560
2020-04-07 00:00:00 3560
2020-04-08 00:00:00 3560
2020-04-09 00:00:00 3515
2020-04-10 00:00:00 3387
2020-04-12 00:00:00 3387


In [35]:
y = []
x = []

In [36]:
for i in date_dict: 
    x.append(i)
    temp_data = date_dict[i] 
    y.append(temp_data[temp_data.Number_of_rooms==1]['price_per_sq_meter'].mean())

In [37]:
type(x[0])

pandas._libs.tslibs.timestamps.Timestamp

In [38]:
px.line(x = x, y=y)